In [1]:
from PIL import Image                   #대구_주민규
import glob
import os

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


In [2]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/train/scissor"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

2711  images to be resized.
2711  images resized.
가위 이미지 resize 완료!


In [3]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/train/rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

2708  images to be resized.
2708  images resized.
바위 이미지 resize 완료!


In [4]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/train/paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

2704  images to be resized.
2704  images resized.
보 이미지 resize 완료!


In [5]:
import numpy as np

def load_data(img_path, number_of_data=8123):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28      #28*28로 리사이징 했기 때문에
    color=3          #흑백이 아니기 때문에 RGB 3색
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):     #노드에 있던 for문 사용 idx를 변화시켜 반복하게 하였다
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/train"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 8123 입니다.
x_train shape: (8123, 28, 28, 3)
y_train shape: (8123,)


In [6]:
import tensorflow as tf
from tensorflow import keras

model=keras.models.Sequential()#시퀀스는 class keras의 models이용 변수 model에 할당시킨다
model.add(keras.layers.Conv2D(25, (3,3), activation='relu', input_shape=(28,28,3)))#흑백이 아니므로 RPB 3
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))          #다양한 특징을 고려하는 변수나 알고리즘의 복잡함을 결정하는 변수로 
model.add(keras.layers.MaxPooling2D((2,2)))                           #적절한 변수의 값을 찾으려 했으나 할때마다 값이 너무 달라져서 포기
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))#가위바위보 3개

print('Model에 추가된 Layer 개수: ', len(model.layers))
model.summary()

Model에 추가된 Layer 개수:  7
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 25)        700       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 25)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        14464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                102464    
_________________________________________________________________
dense_1 (Dense)              (No

In [7]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=25)#epochs20=0.68/25=0.7090/30=0.6842 구간을 세세하게 나누진 못했지만 25이상부터 오버피팅이 일어난다고 가정

Epoch 1/25
254/254 [==============================] - 3s 3ms/step - loss: 0.8506 - accuracy: 0.5910
Epoch 2/25
254/254 [==============================] - 1s 3ms/step - loss: 0.4814 - accuracy: 0.8105
Epoch 3/25
254/254 [==============================] - 1s 3ms/step - loss: 0.3196 - accuracy: 0.8818
Epoch 4/25
254/254 [==============================] - 1s 3ms/step - loss: 0.2384 - accuracy: 0.9142
Epoch 5/25
254/254 [==============================] - 1s 3ms/step - loss: 0.1600 - accuracy: 0.9424
Epoch 6/25
254/254 [==============================] - 1s 3ms/step - loss: 0.1225 - accuracy: 0.9594
Epoch 7/25
254/254 [==============================] - 1s 3ms/step - loss: 0.1032 - accuracy: 0.9658
Epoch 8/25
254/254 [==============================] - 1s 3ms/step - loss: 0.0708 - accuracy: 0.9785
Epoch 9/25
254/254 [==============================] - 1s 3ms/step - loss: 0.0489 - accuracy: 0.9874
Epoch 10/25
254/254 [==============================] - 1s 3ms/step - loss: 0.0333 - accuracy: 0.9915

In [8]:
def load_data(img_path, number_of_data=2100):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28                           #위의 train데이터를 만드는 방법과 같다 다만 data 개수나 image_path정도가 달라짐
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("검사 데이터(x_test)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/test"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화 
  #정규화를 한 이유는 어디서 시작하든 쉽게 최적값에 도달할 수 있으며, 학습률을 상대적으로 높여서 사용할 수 있게 때문에 빠르게 훈련시킬 수 있다. 
print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

검사 데이터(x_test)의 이미지 개수는 2100 입니다.
x_test shape: (2100, 28, 28, 3)
y_test shape: (2100,)


In [10]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

#Conv2D:32=0.7090/25=0.7166/20=0.6490
#Conv2D:64=0.7166/50=0.6661/70=0.6666/60= 0.6719 이런식으로 해보려 했으나 할때마다 값이 달라 한것중에 제일 높았던 상태인 
#epoch 25/25/64/64 Dense는 하려다가 할때마다 값이 달라지는것을 알고 하지 않았습니다.

66/66 - 0s - loss: 3.1028 - accuracy: 0.6590
test_loss: 3.102796792984009 
test_accuracy: 0.6590476036071777


In [ ]:
#처음에는 너무 확률이 말도 안되게 낮게 나와서 코딩의 실수인줄 알고 다른것을 계속 건드려 보고 찾아봤는데 변화가 없었다 그래서 거의 
#포기 직전이였는데 다른 사람들 데이터를 보니 나의 데이터 보다 몇배는 훨씬 많은것을 보고 데이터량을 확연하게 늘렸더니 정상적인 값이
#나왔다. 그리고 좀 더 확률을 높이려 딥러닝의 변수를 다르게 하여 실행시켜 봤는데 변수의 변화보다 그냥 할때마다 달라지는걸 알고 거기서 멈
#추고 가장 높은값이 나왔던 변수로 고정시키고 종료하였다.